In [1]:
import gym
import tensorflow as tf
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline
# base code from udacity-deep-learning/reinforcement/Q-learning-cart.ipynb

In [20]:
# Create new cart pole environment
env = gym.make('CartPole-v1')
state = env.reset()
print(state)

[2017-05-22 17:39:33,840] Making new env: CartPole-v1


[ -3.41691972e-02   4.36618228e-02   2.24535334e-02  -6.69160486e-05]


In [37]:
# Create class QNetwork
class QNetwork:
    def __init__(self, \
                 learning_rate=0.01, \
                 state_size=4, 
                 action_size=2, \
                 hidden_size=10, \
                 hidden_layers=2, \
                 name='QNetwork'):
        
        # create Q Network
        with tf.variable_scope(name):
            
            self.inputs_ = tf.placeholder(tf.float32, \
                                          [None, state_size], \
                                          name='inputs')

            # placeholder for actions, to be one-hot encoded next
            self.actions_ = tf.placeholder(tf.int32, \
                                           [None], \
                                           name='actions')

            # one hot encode actions
            one_hot_actions = tf.one_hot(self.actions_, \
                                         action_size)

            # placeholder for target Qs
            self.targetQs_ = tf.placeholder(tf.float32, \
                                            [None], \
                                            name='target')

                
            # ReLU hidden layers
            self.fc1 = tf.layers.dense(self.inputs_, \
                                        hidden_size,\
                                        activation=tf.nn.relu,\
                                        kernel_initializer=tf.contrib.layers.xavier_initializer())
            
            if hidden_layers == 1:
                out_layer = self.fc1
            else:
                
                self.fc2 = tf.layers.dense(self.fc1, hidden_size,\
                                            activation=tf.nn.relu,\
                                            kernel_initializer=tf.contrib.layers.xavier_initializer())
                if hidden_layers == 2:
                    out_layer = self.fc2
                else:
                    self.fc3 = tf.layers.dense(self.fc2, hidden_size,\
                                            activation=tf.nn.relu,\
                                            kernel_initializer=tf.contrib.layers.xavier_initializer())
                    out_layer = self.fc3

            # Linear output layer
            self.output = tf.layers.dense(out_layer, action_size, \
                                          activation=None,\
                                          kernel_initializer=tf.contrib.layers.xavier_initializer())
            
            ### Train with loss (targetQ - Q)^2
            # output has length 2, for two actions. This next line chooses
            # one value from output (per row) according to the one-hot encoded actions.
            self.Q = tf.reduce_sum(tf.multiply(self.output, one_hot_actions), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.targetQs_ - self.Q))
            self.opt = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [38]:
# create memory class for storing previous experiences
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
            
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

In [39]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [40]:
def initialize_memory_rand_states(memory_size=10000,pretrain_length=20):
    # Initialize the simulation
    state = env.reset()
    
    memory = Memory(max_size=memory_size)

    # Make a bunch of random actions and store the experiences
    for ii in range(pretrain_length):

        # Make a random action
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)

        if done:
            # The simulation fails so no next state
            next_state = np.zeros(state.shape)
            # Add experience to memory
            memory.add((state, action, reward, next_state))

            # Start new episode
            state = env.reset()

        else:
            # Add experience to memory
            memory.add((state, action, reward, next_state))
            state = next_state
            
    return memory

In [41]:
def train_q_network(train_episodes=500,\
                   gamma=0.99,\
                   explore_start=1.0,\
                   explore_stop=0.01,\
                   decay_rate=0.0001,\
                   hidden_size=64,\
                   hidden_layers=2,\
                   learning_rate=0.0001,\
                   memory_size=10000,\
                   batch_size=20,\
                   max_steps=500,\
                   verbose=True):
    
    mainQN = QNetwork(name='main', hidden_size=hidden_size, hidden_layers=hidden_layers, learning_rate=learning_rate)
    
    memory = initialize_memory_rand_states(memory_size=memory_size,pretrain_length=batch_size)
    
    # Now train with experiences
    saver = tf.train.Saver()
    rewards_list = []
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        # Initialize variables
        sess.run(tf.global_variables_initializer())

        step = 0
        state = env.reset()
        for ep in range(train_episodes):
            total_reward = 0
            t = 0
            while t < max_steps:
                step += 1
                # Uncomment this next line to watch the training
                # env.render() 

                # Explore or Exploit
                explore_p = explore_stop + (explore_start - explore_stop)*np.exp(-decay_rate*step) 
                if explore_p > np.random.rand():
                    # Make a random action
                    action = env.action_space.sample()
                else:
                    # Get action from Q-network
                    feed = {mainQN.inputs_: state.reshape((1, *state.shape))}
                    Qs = sess.run(mainQN.output, feed_dict=feed)
                    action = np.argmax(Qs)

                # Take action, get new state and reward
                next_state, reward, done, _ = env.step(action)

                total_reward += reward

                if done:
                    # the episode ends so no next state
                    next_state = np.zeros(state.shape)
                    t = max_steps

                    if verbose:
                        print('Episode: {}'.format(ep),
                              'Total reward: {}'.format(total_reward),
                              'Training loss: {:.4f}'.format(loss),
                              'Explore P: {:.4f}'.format(explore_p))
                    rewards_list.append((ep, total_reward))

                    # Add experience to memory
                    memory.add((state, action, reward, next_state))

                    # Start new episode
                    state = env.reset()
                    # Take one random step to get the pole and cart moving
                    #state, reward, done, _ = env.step(env.action_space.sample())

                else:
                    # Add experience to memory
                    memory.add((state, action, reward, next_state))
                    state = next_state
                    t += 1

                # Sample mini-batch from memory
                batch = memory.sample(batch_size)
                states = np.array([each[0] for each in batch])
                actions = np.array([each[1] for each in batch])
                rewards = np.array([each[2] for each in batch])
                next_states = np.array([each[3] for each in batch])

                # Train network
                target_Qs = sess.run(mainQN.output, feed_dict={mainQN.inputs_: next_states})

                # Set target_Qs to 0 for states where episode ends
                episode_ends = (next_states == np.zeros(states[0].shape)).all(axis=1)
                target_Qs[episode_ends] = (0, 0)

                targets = rewards + gamma * np.max(target_Qs, axis=1)

                loss, _ = sess.run([mainQN.loss, mainQN.opt],
                                    feed_dict={mainQN.inputs_: states,
                                               mainQN.targetQs_: targets,
                                               mainQN.actions_: actions})

        saver.save(sess, "checkpoints/cartpole.ckpt")
        return rewards_list, mainQN, saver

In [8]:
def plot_rewards(rewards_list):
    eps, rews = np.array(rewards_list).T
    smoothed_rews = running_mean(rews, 10)
    plt.plot(eps[-len(smoothed_rews):], smoothed_rews)
    plt.plot(eps, rews, color='grey', alpha=0.3)
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')

In [42]:
def test_q_network(mainQN, saver, test_episodes=20, test_max_steps=500, render=True):

    env.reset()
    avg_rewards = 0.
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
   
        state = env.reset()
        for ep in range(test_episodes):
            t = 0
            while t < test_max_steps:
                if render:
                    env.render() 

                # Get action from Q-network
                feed = {mainQN.inputs_: state.reshape((1, *state.shape))}
                Qs = sess.run(mainQN.output, feed_dict=feed)
                action = np.argmax(Qs)

                # Take action, get new state and reward
                next_state, reward, done, _ = env.step(action)
                
                avg_rewards = avg_rewards + reward / test_episodes
                if done:
                    t = test_max_steps
                    state = env.reset()
                    # Take one random step to get the pole and cart moving
                    #state, reward, done, _ = env.step(env.action_space.sample())

                else:
                    state = next_state
                    t += 1
                    
    return avg_rewards

In [43]:
def test_and_train_qnetwork(train_episodes=500,\
                   gamma=0.99,\
                   explore_start=1.0,\
                   explore_stop=0.01,\
                   decay_rate=0.0001,\
                   hidden_size=64,\
                   hidden_layers=2,\
                   learning_rate=0.0001,\
                   memory_size=10000,\
                   batch_size=20,\
                   test_episodes=20,\
                   render=False,\
                   verbose=True):
    
    # reset graph
    tf.reset_default_graph()

    # train q-network
    rewards_list, mainQN, saver = train_q_network(train_episodes = train_episodes, \
                                                  gamma=gamma,\
                                                  explore_start=explore_start,\
                                                  explore_stop=explore_stop,\
                                                  decay_rate=decay_rate,\
                                                  hidden_size=hidden_size,\
                                                  hidden_layers=hidden_layers,\
                                                  learning_rate=learning_rate,\
                                                  memory_size=memory_size,\
                                                  batch_size=batch_size,\
                                                  verbose=False)

    if verbose:
        # plot training
        plot_rewards(rewards_list)
    
    avg_train_rewards = np.sum([each[1] for each in rewards_list]) / len(rewards_list)
    if verbose:
        print('average training reward = ',avg_train_rewards)

    # test q-network
    avg_test_rewards = test_q_network(mainQN, saver, test_episodes=test_episodes, render=render)
    if verbose:
        print('average test reward = ', avg_test_rewards)
    
    return avg_test_rewards


In [44]:
tf.reset_default_graph()
train_q_network()

InternalError: Blas GEMM launch failed : a.shape=(20, 4), b.shape=(4, 64), m=20, n=64, k=4
	 [[Node: main/dense/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_main/inputs_0/_1, main/dense/kernel/read)]]

Caused by op 'main/dense/MatMul', defined at:
  File "C:\Users\drbur\AppData\Local\Continuum\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\drbur\AppData\Local\Continuum\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\drbur\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\drbur\AppData\Local\Continuum\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_instance
    app.start()
  File "C:\Users\drbur\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
 

In [199]:
# test implementation
average_rewards = test_and_train_qnetwork(train_episodes=100, verbose=False)
print('average test reward = ', average_rewards)

INFO:tensorflow:Restoring parameters from checkpoints\cartpole.ckpt


[2017-05-22 15:54:17,537] Restoring parameters from checkpoints\cartpole.ckpt


average test reward =  9.249999999999996


In [ ]:
train_eps = 500
verb = False
gamma = 0.99
exp_start=1.0
exp_stop=0.1
decay_rate=0.0001
hidden_size=[16,32,64]
hidden_layers=[1,2,3]
learning_rate=[0.01,0.001,0.0001]
batch_size=[10,20,40]
num_averages = 2
results = []

for hs in hidden_size:
    for hl in hidden_layers:
        for lr in learning_rate:
            for bs in batch_size:
                train_params_name = 'hs='+str(hs)+'_hl='+str(hl)+'_lr='+str(lr)+'_bs='+str(bs)
                average_rewards = 0.
                for i in range(num_averages):
                    average_rewards += test_and_train_qnetwork(train_episodes=train_eps,\
                                           gamma=gamma,\
                                           explore_start=exp_start,\
                                           explore_stop=exp_stop,\
                                           decay_rate=decay_rate,\
                                           hidden_layers=hl,\
                                           hidden_size=hs,\
                                           learning_rate=lr,\
                                           batch_size=bs,\
                                           verbose=verb)
                average_rewards = average_rewards / num_averages
                results.append([train_params_name+' '+str(average_rewards)])
                clear_output()
                for each in results:
                    print(each)

In [ ]:
env.close()

In [31]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/gpu:0', '/gpu:1']